In [1]:
# from google.colab import drive
# drive.mount('/content/drive' , force_remount= True)

In [2]:
#!unzip /content/drive/MyDrive/DNA_Data_model/Prepared_Data.csv.zip

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
#from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , precision_score , recall_score , f1_score , confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import pickle
import joblib

In [4]:
df_ = pd.read_pickle("../data/processed/final_processed_data.pkl")

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
df = df_.copy()

In [7]:
df.head()

,Name,Parent_full_DNA_Seq,Child_full_DNA_Seq,ParentM,ParentF,target
0,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
1,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
2,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...,NaN,NaN,1
3,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,1
4,A1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58784 entries, 0 to 14999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name                 58784 non-null  object
 1   Parent_full_DNA_Seq  58784 non-null  object
 2   Child_full_DNA_Seq   43784 non-null  object
 3   ParentM              21892 non-null  object
 4   ParentF              21892 non-null  object
 5   target               58784 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 3.1+ MB


In [9]:
df['target'].unique()

array([0, 1])

In [10]:
db = df.tail(100)

In [11]:
df.drop(df.tail(1000).index,inplace=True)
df

,Name,Parent_full_DNA_Seq,Child_full_DNA_Seq,ParentM,ParentF,target
0,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
1,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0
2,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...,NaN,NaN,1
3,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,1
4,A1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0
...,...,...,...,...,...,...
13995,E13995,CTCCGTCGACGCTTTAGGGCCATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
13996,E13996,CTCCGTCGAAGCTTTAGGGACATAGATGGGAGCTCTGGTTCCCGTG...,NaN,NaN,NaN,0
13997,E13997,CTCCGTCGACGCTTTAAGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0
13998,E13998,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,NaN,0


In [12]:
#db.to_csv("db_data.csv")

In [13]:
df['Parent_full_DNA_Seq'] = df['Parent_full_DNA_Seq'].astype('str')
df['Child_full_DNA_Seq'] = df['Child_full_DNA_Seq'].astype('str')

In [14]:
df_pre = df[['Parent_full_DNA_Seq','Child_full_DNA_Seq','target']]

In [15]:
x = df_pre.drop('target',axis=1)
y = df_pre['target']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [16]:
x.shape , y.shape

((55784, 2), (55784,))

In [17]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [18]:
vectorizer = CountVectorizer(analyzer = 'char' ,ngram_range=(3,3) , min_df=1)
X_ch = vectorizer.fit_transform(X_train['Child_full_DNA_Seq'])
# x_ch >>>>>>>> vectorizer
X_p = vectorizer.fit_transform(X_train['Parent_full_DNA_Seq'])
# X_p >>>>>>>> vectorizer_0
kmer_embeddings_c = X_ch.toarray()
kmer_embeddings_p = X_p.toarray()

In [19]:
filename = 'kmer_model.sav'
joblib.dump(vectorizer , filename)

['kmer_model.sav']

In [20]:
kmer_embeddings_p = np.pad(kmer_embeddings_p, ((0, 0), (0, 1)), 'constant')
for i in range (0 , len(kmer_embeddings_c[0])):
    X_train['child_gene_k_' + str(i)] = [kmer_embeddings_c[j][i] for j in range (0, len(X_train))]
    X_train['parent_gene_k_' + str(i)] = [kmer_embeddings_p[j][i] for j in range (0, len(X_train))]

/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_6372/699562758.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train['parent_gene_k_' + str(i)] = [kmer_embeddings_p[j][i] for j in range (0, len(X_train))]
/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_6372/699562758.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train['child_gene_k_' + str(i)] = [kmer_embeddings_c[j][i] for j in range (0, len(X_train))]


In [21]:
X_train = X_train.drop(['Parent_full_DNA_Seq','Child_full_DNA_Seq'],axis=1)

In [22]:
param_grid = {
    'learning_rate': [0.1, 0.3, 0.5 , 0.7],
    'n_estimators': [5000, 10000, 15000, 20000],
    'max_depth': [5, 10, 15, 20]
}

In [23]:
xgb_model = XGBClassifier()
grid_search = GridSearchCV(xgb_model, param_grid, cv=3, scoring='accuracy', n_jobs=-1)

In [24]:
# Working with test data
X_t_c= vectorizer.transform(X_test['Child_full_DNA_Seq'])
# X_t_c vect
X_t_p = vectorizer.transform(X_test['Parent_full_DNA_Seq'])
# X_t_p >>>>>>>> vectorizer_0
kmer_embeddings_child_test = X_t_c.toarray()
kmer_embeddings_parent_test = X_t_p.toarray()

In [25]:
for i in range (0, len(kmer_embeddings_child_test[0])):
    X_test['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(X_test))]
    X_test['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(X_test))]

/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_6372/477141918.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['parent_gene_k_'+str(i)] = [kmer_embeddings_parent_test[j][i] for j in range (0, len(X_test))]
/var/folders/3z/w_2xtxjs7517xwd76c815h5c0000gn/T/ipykernel_6372/477141918.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test['child_gene_k_'+str(i)] = [kmer_embeddings_child_test[j][i] for j in range (0, len(X_test))]


In [26]:
X_train

,child_gene_k_0,parent_gene_k_0,child_gene_k_1,parent_gene_k_1,child_gene_k_2,parent_gene_k_2,child_gene_k_3,parent_gene_k_3,child_gene_k_4,parent_gene_k_4,...,child_gene_k_60,parent_gene_k_60,child_gene_k_61,parent_gene_k_61,child_gene_k_62,parent_gene_k_62,child_gene_k_63,parent_gene_k_63,child_gene_k_64,parent_gene_k_64
2321,1114,1095,1182,1156,1156,1180,1054,1028,1120,1102,...,1167,1078,1079,1230,1178,1158,1118,1124,1123,0
8101,1093,1120,1147,1127,1153,1161,1056,1028,1092,1157,...,1120,1066,1059,1202,1221,1140,1148,1132,1123,0
2754,1012,1049,1195,1117,1162,1153,1004,1058,1111,1098,...,1199,1047,1060,1197,1214,1127,1126,1126,1091,0
15740,1108,1068,1081,1083,1167,1152,1034,1028,1048,1087,...,1205,1027,1064,1227,1208,1179,1162,1112,1085,0
2617,1053,1040,1126,1095,1181,1153,1040,1050,1126,1128,...,1148,1107,1044,1192,1190,1140,1135,1135,1168,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0,1135,0,1254,0,1203,0,1100,0,1182,...,0,1132,0,1176,0,1146,0,1097,0,0
12559,0,1103,0,1239,0,1188,0,1144,0,1155,...,0,1068,0,1135,0,1214,0,1156,0,0
18266,1043,1043,1176,1136,1173,1174,1042,1032,1169,1192,...,1162,1058,1068,1183,1199,1123,1105,1092,1076,0
860,1001,1044,1130,1151,1187,1145,1019,1040,1137,1104,...,1178,1088,1070,1184,1202,1142,1126,1152,1156,0


In [27]:
X_train = X_train.drop(['child_gene_k_64','parent_gene_k_64'],axis=1)

In [28]:
X_test.shape , X_train.shape

((16736, 130), (39048, 128))

In [29]:
X_test.drop(['Parent_full_DNA_Seq','Child_full_DNA_Seq'],axis=1,inplace=True)

In [30]:
X_test

,child_gene_k_0,parent_gene_k_0,child_gene_k_1,parent_gene_k_1,child_gene_k_2,parent_gene_k_2,child_gene_k_3,parent_gene_k_3,child_gene_k_4,parent_gene_k_4,...,child_gene_k_59,parent_gene_k_59,child_gene_k_60,parent_gene_k_60,child_gene_k_61,parent_gene_k_61,child_gene_k_62,parent_gene_k_62,child_gene_k_63,parent_gene_k_63
5874,1020,1052,1153,1136,1136,1137,1038,1068,1157,1164,...,1145,1176,1029,1025,1196,1192,1123,1100,1047,1050
6992,1028,1056,1141,1143,1152,1173,992,1007,1124,1149,...,1147,1159,1069,1077,1159,1132,1167,1117,1138,1122
16861,1066,1091,1153,1171,1213,1196,1027,988,1112,1081,...,1158,1171,1031,1046,1195,1215,1163,1171,1032,1124
1698,0,1136,0,1218,0,1139,0,1046,0,1152,...,0,1208,0,1147,0,1223,0,1221,0,1115
19771,1060,1077,1171,1188,1158,1180,1040,1007,1075,1090,...,1146,1159,1105,1064,1223,1186,1133,1150,1157,1172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13182,1024,999,1122,1158,1161,1164,987,982,1072,1110,...,1219,1177,1056,1054,1190,1177,1193,1176,1108,1105
13424,0,1168,0,1217,0,1201,0,1147,0,1207,...,0,1188,0,1119,0,1185,0,1164,0,1094
10599,992,1037,1111,1142,1163,1187,999,973,1102,1107,...,1200,1182,1093,1059,1192,1167,1129,1175,1112,1143
17685,1035,1095,1163,1130,1170,1159,1004,1002,1131,1165,...,1219,1180,1072,1019,1202,1200,1121,1110,1056,1108


In [31]:
X_train

,child_gene_k_0,parent_gene_k_0,child_gene_k_1,parent_gene_k_1,child_gene_k_2,parent_gene_k_2,child_gene_k_3,parent_gene_k_3,child_gene_k_4,parent_gene_k_4,...,child_gene_k_59,parent_gene_k_59,child_gene_k_60,parent_gene_k_60,child_gene_k_61,parent_gene_k_61,child_gene_k_62,parent_gene_k_62,child_gene_k_63,parent_gene_k_63
2321,1114,1095,1182,1156,1156,1180,1054,1028,1120,1102,...,1193,1173,1167,1078,1079,1230,1178,1158,1118,1124
8101,1093,1120,1147,1127,1153,1161,1056,1028,1092,1157,...,1238,1129,1120,1066,1059,1202,1221,1140,1148,1132
2754,1012,1049,1195,1117,1162,1153,1004,1058,1111,1098,...,1246,1204,1199,1047,1060,1197,1214,1127,1126,1126
15740,1108,1068,1081,1083,1167,1152,1034,1028,1048,1087,...,1225,1201,1205,1027,1064,1227,1208,1179,1162,1112
2617,1053,1040,1126,1095,1181,1153,1040,1050,1126,1128,...,1213,1116,1148,1107,1044,1192,1190,1140,1135,1135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,0,1135,0,1254,0,1203,0,1100,0,1182,...,0,1141,0,1132,0,1176,0,1146,0,1097
12559,0,1103,0,1239,0,1188,0,1144,0,1155,...,0,1153,0,1068,0,1135,0,1214,0,1156
18266,1043,1043,1176,1136,1173,1174,1042,1032,1169,1192,...,1239,1197,1162,1058,1068,1183,1199,1123,1105,1092
860,1001,1044,1130,1151,1187,1145,1019,1040,1137,1104,...,1230,1202,1178,1088,1070,1184,1202,1142,1126,1152


In [32]:

'''
this is new normalizing to Scaling the data from range (1000) to (0 , 1 )
'''
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [33]:
train_x = X_train.values
test_x = X_test.values

In [34]:
grid_search.fit(train_x, y_train)

KeyboardInterrupt: 

In [ ]:
best_params = grid_search.best_params_
best_xgb_model = grid_search.best_estimator_

In [ ]:
# test_x = test_x.reshape(-1, 130)

In [ ]:
y_pred_xgb = best_xgb_model.predict(test_x)

In [ ]:
y_pred_xgb.shape

In [ ]:
# svm = SVC(kernel='rbf' , C = .5 , gamma ='scale')
# svm.fit(X_train , y_train)
# y_pred_svm = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred_svm)


In [ ]:
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
# Print the results
print("XGBoost Model Evaluation with Hyperparameter Tuning:")
print(f"Accuracy: {accuracy_xgb * 100:.2f}%")
print(f"Precision: {precision_xgb * 100:.2f}%")
print(f"Recall: {recall_xgb * 100:.2f}%")
print(f"F1 Score: {f1_xgb * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix_xgb)
print("\n")
print("Best Hyperparameters:", best_params)

# Saving Model

In [ ]:
# # saving Model
# with open('xgb_Model.pkl' , 'wb') as f:
#   pickle.dump(model_xgb, f)

## K-mer saving

In [ ]:
# filename = 'kmer_model.sav'
# joblib.dump(vectorizer , filename)